# Cvičenie 4: Policy Iteration

Na minulom cvičení ste videli, ako sa dajú vyriešiť Markovovské rozhodovacie procesy pomocou Bellmanových rovníc. Od tohto týždňa sa pozrieme na rôzne metódy učenia posilňovaním, pričom ich budeme aplikovať na rovnaký gridworld problém pre porovnávanie. Prvá metóda bude práve Policy Iteration.

## Policy Iteration

Policy Iteration pozostáva z iteratívnej aplikácie dvoch krokov, a to vyhodnocovanie (*policy evaluation*) a zlepšovanie (*policy improvement*). Pseudokód algoritmu nájdete na obrázko nižšie.

<img src="sources/lab04/policy_iteration.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Prediskutujte význam jednotlivých operácií a krokov a následne aplikujte algoritmus na ukážkový príklad gridworld. Ako ukážku, nižšie nájdete vypracované kroky pre jeden stav.

<img src="sources/lab04/gridworld_example2.jpg" width="300">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v druhom riadku a prvom stĺpci. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Hráč nemôže prejsť cez stenu, v tomto prípade ostane na rovnakej pozícii, ale berie sa, akokeby vykonal krok. Ak hráč sa dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra skončí. Pre ostatné kroky dostane odmenu -1.

Discount factor $\gamma = 0.8$ a $\theta = 1.0$. Začneme s náhodnou politikou $\pi_0$: na každej pozícii existuje rovnaká pravdepodobnosť výberu jednotlivých akcií (okrem cieľovej pozície a pasce).

### 1. Policy Evaluation

Pre stav $s_{11}$ (prvý riadok, prvý stĺpec) vieme vypočítať hodnotu stavu dvomi spôsobmi:

1. iterujeme nad stavovým priestorom a berieme do úvahy pravdepodobnosť, že hráč sa z aktuálnej pozície dostane na druhú pozíciu (ako v pseudokóde vyššie)
2. iterujeme nad priestorom akcií a berieme do úvahy pravdepodobnosť, že sa vyberie daná akcia a pravdepodobnosť toho, že vykonaním akcie sa hráč presunie na istú druhú púozíciu.

Pre prístup 1 existujú tri možné stavy, kam sa hráč reálne môže dostať: $s_{11}$ (akcie N a W), $s_{12}$ (akcia E) a $s_{21}$ (akcia S). Novú hodnotu $V(s_{11})$ potom vieme vypočítať ako:

$V(s_{11}) = p \left ( s_{11}, r | s_{11}, \pi_0 \right ) \cdot (r + \gamma \cdot V(s_{11})) + p \left ( s_{12}, r | s_{11}, \pi_0 \right ) \cdot (r + \gamma \cdot V(s_{12})) + p \left ( s_{21}, r | s_{11}, \pi_0 \right ) \cdot (r + \gamma \cdot V(s_{21})) $

$V(s_{11}) = 0.5 \cdot (-1 + 0.8 \cdot 0) + 0.25 \cdot (-1 + 0.8 \cdot 0) + 0.25 \cdot (-10 + 0.8 \cdot 0) = -0.5 -0.25 - 2.5 = -3.25$

Alebo pre prístup 2:

$V(s_{11}) = p \left ( a_0 | s_{11}, \pi_0 \right ) \cdot p \left ( s_{11}, r | s_{11}, a_0) \right ) \cdot (r + \gamma \cdot V(s_{11})) + p \left ( a_1 | s_{11}, \pi_0 \right ) \cdot p \left ( s_{11}, r | s_{11}, a_1) \right ) \cdot (r + \gamma \cdot V(s_{11})) + p \left ( a_2 | s_{11}, \pi_0 \right ) \cdot p \left ( s_{12}, r | s_{11}, a_2) \right ) \cdot (r + \gamma \cdot V(s_{12})) + p \left ( a_3 | s_{11}, \pi_0 \right ) \cdot p \left ( s_{13}, r | s_{11}, a_3) \right ) \cdot (r + \gamma \cdot V(s_{13})) $

$V(s_{11}) = 0.25 \cdot 1.0 \cdot (-1 + 0.8 \cdot 0) + 0.25 \cdot 1.0 \cdot (-1 + 0.8 \cdot 0) + 0.25 \cdot 1.0 \cdot (-1 + 0.8 \cdot 0) + 0.25 \cdot 1.0 \cdot (-10 + 0.8 \cdot 0) $
$V(s_{11}) = -0.25 -0.25 -0.25 - 2.5 = -3.25$

Zmena odhadu pre $V(s_{11})$ je $\Delta V(s_{11}) = |0 - 3.25| = 3.25$, čo je väčšie ako $\theta$, takže už teraz je zrejmé, že vyhodnocovanie si budeme musieť zopakovať.

**Úloha:** Vypočítajte odhad hodnoty pre ostatné stavy (okrem $s_{13}$ a $s_{21}$). Použite pritom inplace prístup.

### 2. Policy Improvement

Začiatočná politika $\pi_0$ bola definovaná náhodne, teda pravdepobonosť každej akcie bola $0.25$. Ak uz máme vypočítane odhadované hodnoty stavov, tieto hodnoty vieme použiť pri zlepšovaní politiky. Pre každú akciu vypočítame očakávanú dlhodobú odmenu podľa vzorca:

$v_{\pi}(s, a) = \sum_{s', r} p(s', r|s, a) \cdot (r + \gamma \cdot V(s'))$

Teda pre stav $s_{11}$:

$v_{\pi_0}(s_{11}, a_0) = p(s_{11}, r|s_{11}, a_0) \cdot (r + \gamma \cdot V(s_{11})) = 1.0 \cdot (-1 + 0.8 \cdot -3.25) = -4.90592 $

$v_{\pi_0}(s_{11}, a_1) = p(s_{12}, r|s_{11}, a_1) \cdot (r + \gamma \cdot V(s_{12})) = 1.0 \cdot (-1 + 0.8 \cdot 0.19856) = -0.841152 $

$v_{\pi_0}(s_{11}, a_2) = p(s_{21}, r|s_{11}, a_2) \cdot (r + \gamma \cdot V(s_{21})) = 1.0 \cdot (-10 + 0.8 \cdot 0) = -10 $

$v_{\pi_0}(s_{11}, a_3) = p(s_{11}, r|s_{11}, a_3) \cdot (r + \gamma \cdot V(s_{11})) = 1.0 \cdot (-1 + 0.8 \cdot -4.8824 = -4.90592$

pre jednotlivé akcie. Suma sa nám zjednoduší, keďže máme deterministický svet, kde pri vykonaní akcie prostredie sa môže aktualizovať len do jedného stavu. Politiku pre stav teda vieme aktualizovať (hodnoty zodpovedajú pravdepodobnosti výberu danej akcie N, E, S, W):

$\pi_1(s_{11}) = \{ 0.0, \ 1.0, \ 0.0, \ 0.0 \}$

**Úloha:** Vypočítajte všetky očakávané odmeny pre dvojice stav-akcia (okrem stavov $s_{13}$ a $s_{21}$) a na základe hodnôt aktualizujte politiku na $\pi_1$.

## Implementácia

Vaše výpočty overte na [ukážkovej implementácii metódy Policy Iteration](sources/lab04/policy_iteration.py) a spozorujte, ako sa metóda dopracuje k optimálnej politike.

## Domáca úloha

Ukážkové riešenie používa inplace stratégiu pre aktualizáciu údajov. Upravte ho tak, aby používalo sweep stratégiu, následne porovnajte funkcionalitu dvoch prístupov. Vaše poznámky napíšte do komentárov ku kódu.